In [41]:
import os
import json
import pandas as pd
import torch
from torchvision.io import read_image
from torchvision.transforms import v2 as T

In [42]:
path_base = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),'Data')
images_dir_root = os.path.join(path_base,'data_sorted','frames')

In [43]:
p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14, p15, p16 = ([] for i in range(16))
for dir in os.listdir(images_dir_root):
    if 'p1_' in dir:
        p1.append(os.path.join(images_dir_root, dir))
    elif 'p2' in dir:
        p2.append(os.path.join(images_dir_root, dir))
    elif 'p3' in dir:
        p3.append(os.path.join(images_dir_root, dir))
    elif 'p4' in dir:
        p4.append(os.path.join(images_dir_root, dir))
    elif 'p5' in dir:
        p5.append(os.path.join(images_dir_root, dir))
    elif 'p6' in dir:
        p6.append(os.path.join(images_dir_root, dir))
    elif 'p7' in dir:
        p7.append(os.path.join(images_dir_root, dir))
    elif 'p8' in dir:
        p8.append(os.path.join(images_dir_root, dir))
    elif 'p9' in dir:
        p9.append(os.path.join(images_dir_root, dir))
    elif 'p10' in dir:
        p10.append(os.path.join(images_dir_root, dir))
    elif 'p11' in dir:
        p11.append(os.path.join(images_dir_root, dir))
    elif 'p12' in dir:
        p12.append(os.path.join(images_dir_root, dir))
    elif 'p13' in dir:
        p13.append(os.path.join(images_dir_root, dir))
    elif 'p14' in dir:
        p14.append(os.path.join(images_dir_root, dir))
    elif 'p15' in dir:
        p15.append(os.path.join(images_dir_root, dir))
    elif 'p16' in dir:
        p16.append(os.path.join(images_dir_root, dir))
video_dirs = [p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14, p15, p16]

In [44]:
report_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),'cleaned.csv')
df = pd.read_csv(report_path)
df.reset_index(drop=True, inplace=True)

analyzed_path = os.path.join(path_base, 'analyzed_imgs')

In [45]:
videos = video_dirs[0]
frames = os.listdir(videos[0])
os.path.join(videos[0], frames[1337])

'c:\\Users\\admin\\Documents\\Arthroskopie\\Data\\data_sorted\\frames\\p1_v1\\frame_001337.PNG'

In [46]:
def get_transform(train, h_flip, v_flip):
    transforms = []
    if train:
        transforms.append(T.RandomHorizontalFlip(h_flip))
        transforms.append(T.RandomVerticalFlip(v_flip))
    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.ToPureTensor())
    return T.Compose(transforms)

In [47]:
def analyze_image(video, id, model, device):
    image = read_image(os.path.join(video, id))
    eval_transform = get_transform(train=False, h_flip=0, v_flip=0)
    with torch.no_grad():
        y = eval_transform(image)
        y = y[:3, ...].to(device)
        predictions = model([y, ])
        pred = predictions[0]
    
    return pred

In [ ]:
def generate_frames(patient, frames_to_accumalate, model, device):
    analyzed_images = []
    videos = video_dirs[patient]
    for x in videos:
        frames = os.listdir(x)
        counter = 0
        while counter < len(frames):
            one_hot = torch.zeros(32)
            scores = torch.zeros(32)
            for i in range(frames_to_accumalate):
                if counter+i < len(frames):
                    pred = analyze_image(x, frames[(counter+i)], model, device)
                    for z in range(len(pred["labels"])):
                        cat = pred["labels"][z].item()-1
                        score = pred["scores"][z].item()
                        if score >= 0.75:
                            if one_hot[cat] == 0:
                                one_hot[cat] = 1
                                scores[cat] = score
                            else:
                                if scores[cat] < score:
                                    scores[cat] = score
                else:
                    break                    
        
            fin_img=torch.cat([torch.zeros(15), one_hot, scores])
            analyzed_images.append(fin_img)
            counter += frames_to_accumalate
            
        print("Finished: ", x)
    return analyzed_images

In [49]:
from pickle import  load
import os
import pandas as pd
import torch

with open(os.path.dirname(os.getcwd()) + "\\Object_Detection\\dataloaders\\names_to_annotation.pkl", "rb") as input_file:
   names_to_annotation = load(input_file)

In [50]:
report_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),'cleaned.csv')
df = pd.read_csv(report_path)

In [ ]:
df['ICD-Code'][0:16]

In [ ]:
icd_dict = {}
counter = 0

for row in df['ICD-Code'][0:16]:
    try:
        for code in row.split(','):  
            cleaned_code = code.strip()
            if cleaned_code not in icd_dict.keys():
                icd_dict[cleaned_code] = counter
                counter +=1
    except:
        print('Skipped: ' , row)

In [ ]:
icd_dict

In [ ]:
icpm_dict = {}
counter = 0

for row in df['ICPM-Code'][0:16]:
    try:
        for code in row.split(','):  
            cleaned_code = code.strip()
            if cleaned_code not in icpm_dict.keys():
                icpm_dict[cleaned_code] = counter
                counter +=1
    except:
        print('Skipped: ' , row)

In [ ]:
icpm_dict

In [56]:
def tokenize_codes(icd, icpm):
    icd_codes = torch.zeros(7)
    icpm_codes = torch.zeros(8)

    try:
        icd_list = icd.split(',')
        for code in icd_list:
            cleaned_code = code.strip()
            pos = icd_dict[cleaned_code]
            icd_codes[pos] = 1

        icpm_list = icpm.split(',')
        for code in icpm_list:
            cleaned_code = code.strip()
            pos = icpm_dict[cleaned_code]
            icpm_codes[pos] = 1
    except:
        print('Skipped NaN')
    
    return torch.cat([icd_codes, icpm_codes, torch.zeros(82)])

In [ ]:
from pickle import  dump

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torch.load(os.path.dirname(os.getcwd()) + "\\Object_Detection\\Models\\model_whole_no_empty.pt", weights_only=False)
model.to(device)

for a in range(len(video_dirs)):
    
    series = df.loc[a]
    icd_icpm = tokenize_codes(series.iloc[2], series.iloc[4])
    
    frames = generate_frames(a, 30, model, device)

    data = {'Ablauf':series.iloc[1],'icd_icpm': icd_icpm, 'frames': frames}
    
    file_name = "p{}.pkl".format(a+1)
    print(file_name)
    with open(os.path.join(analyzed_path, file_name), 'wb') as f:
        dump(data, f)

In [ ]:
from pickle import  load

test_path = os.path.join(path_base, 'analyzed_imgs')
dir = os.listdir(test_path)

for file in dir:
   with open(os.path.join(test_path, file), "rb") as input_file:
      input = load(input_file)
      list = input['frames']
   print(f'{file}: {len(list)}')